In [1]:
# # # Parameters
DT = "2024-06-17"   

In [2]:
from skt.gcp import bq_to_df, get_bigquery_client, df_to_bq_table
from pyspark.sql.functions import pandas_udf, udf, split, explode, collect_list, col, concat_ws, lit
from pyspark.sql.types import StringType
import pandas as pd
from datetime import datetime, date, timedelta

In [3]:
# DT = "2024-06-02"
current_date = datetime.strptime(DT, "%Y-%m-%d")
DT_threshold1 = (current_date - timedelta(days=30)).strftime("%Y-%m-%d")
DT_threshold2 = (current_date - timedelta(days=7)).strftime("%Y-%m-%d")

print("DT : ", DT)
print("DT_threshold : ", DT_threshold1)
print("DT_threshold : ", DT_threshold2)

DT :  2024-06-17
DT_threshold :  2024-05-18
DT_threshold :  2024-06-10


## Adot user 한정해서 Table 생성 후 각 도메인 별 Text화 작업 수행

### (1) Adot 제외한 도메인 TEXT 화

In [4]:
total_data = bq_to_df("""
    WITH
    
    adot_user_data AS (
        SELECT distinct svc_mgmt_num,
                        luna_id,
                        ci,
                        gender,
                        age
        FROM (
            SELECT * 
            FROM x1112904.adot_agr_meta_daily
            WHERE dt = (SELECT max(dt) FROM x1112904.adot_agr_meta_daily)
            AND status IS NULL
            
            UNION DISTINCT
            SELECT *
            FROM x1112904.adot_agr_meta_daily
            WHERE dt = (SELECT max(dt) FROM x1112904.adot_agr_meta_daily)
            AND status = 'NORMAL'
        )
        WHERE svc_mgmt_num is not null
    )
    
    SELECT adot.*, store11.* EXCEPT(svc_mgmt_num, dt), cdr.* EXCEPT(svc_mgmt_num, dt), prod1.* EXCEPT(svc_mgmt_num, dt), subsc.* EXCEPT(svc_mgmt_num, dt), 
    tdeal.* EXCEPT(svc_mgmt_num, dt), tmap.* EXCEPT(svc_mgmt_num, dt), tmbr.* EXCEPT(svc_mgmt_num, dt), xdr.* EXCEPT(svc_mgmt_num, dt), xtr.* EXCEPT(svc_mgmt_num, dt)
    FROM adot_user_data as adot
    LEFT JOIN (SELECT * FROM x1113099.one_model_profile_11st WHERE dt = (select max(dt) from x1113099.one_model_profile_11st)) AS store11 ON adot.svc_mgmt_num=store11.svc_mgmt_num
    LEFT JOIN (SELECT * FROM x1113099.one_model_profile_cdr WHERE dt = (select max(dt) from x1113099.one_model_profile_cdr)) AS cdr ON adot.svc_mgmt_num=cdr.svc_mgmt_num
    LEFT JOIN (SELECT * FROM x1113099.one_model_profile_prod1 WHERE dt = (select max(dt) from x1113099.one_model_profile_prod1)) AS prod1 ON adot.svc_mgmt_num=prod1.svc_mgmt_num
    LEFT JOIN (SELECT * FROM x1113099.one_model_profile_subsc WHERE dt = (select max(dt) from x1113099.one_model_profile_subsc)) AS subsc ON adot.svc_mgmt_num=subsc.svc_mgmt_num
    LEFT JOIN (SELECT * FROM x1113099.one_model_profile_tdeal WHERE dt = (select max(dt) from x1113099.one_model_profile_tdeal)) AS tdeal ON adot.svc_mgmt_num=tdeal.svc_mgmt_num
    LEFT JOIN (SELECT * FROM x1113099.one_model_profile_tmap WHERE dt = (select max(dt) from x1113099.one_model_profile_tmap)) AS tmap ON adot.svc_mgmt_num=tmap.svc_mgmt_num
    LEFT JOIN (SELECT * FROM x1113099.one_model_profile_tmbr WHERE dt = (select max(dt) from x1113099.one_model_profile_tmbr)) AS tmbr ON adot.svc_mgmt_num=tmbr.svc_mgmt_num
    LEFT JOIN (SELECT * FROM x1113099.one_model_profile_xdr WHERE dt = (select max(dt) from x1113099.one_model_profile_xdr)) AS xdr ON adot.svc_mgmt_num=xdr.svc_mgmt_num
    LEFT JOIN (SELECT * FROM x1113099.one_model_profile_xtr WHERE dt = (select max(dt) from x1113099.one_model_profile_xtr)) AS xtr ON adot.svc_mgmt_num=xtr.svc_mgmt_num
""")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/18 15:29:23 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
24/06/18 15:29:45 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


In [5]:
total_data1 = total_data.fillna("정보없음")

- 11st : 11번가 관련 정보

In [6]:
def _11st_profile_text(_11st_cat2_prefer, _11st_item_prefer):
    if _11st_item_prefer == "정보없음":
        return ""
    else:
        return f"저는 11번가 쇼핑몰에서 주로 {_11st_cat2_prefer} 카테고리에 관심이 있고, 최근에는 {_11st_item_prefer} 키워드 기반의 상품을 찾아봤어요."

_11st_text_udf = udf(_11st_profile_text, StringType())

In [7]:
total_data1 = total_data1.withColumn("_11st_text", _11st_text_udf(total_data1["_11st_cat2_prefer"], total_data1["_11st_item_prefer"]))

- cdr : 수발신이력

In [8]:
def cdr_profile_text(cdr_cat2_prefer, cdr_item_prefer):
    if cdr_item_prefer == "정보없음":
        return ""
    else:
        return f"저는 주로 {cdr_cat2_prefer} 카테고리에 수발신을 하고, 최근에는 {cdr_item_prefer} 쪽으로 주로 연락했어요."

cdr_text_udf = udf(cdr_profile_text, StringType())

In [9]:
total_data1 = total_data1.withColumn("cdr_text", cdr_text_udf(total_data1["cdr_cat2_prefer"], total_data1["cdr_item_prefer"]))

- prod1 : 요금제 정보 
    - item : 실제 요금제 정보
    - cat1 : 일반 요금제인지 다이렉트 요금제인지
    - cat2 : 어디에서 개통했는지? 기타 티월드 유통망 고객센터

In [10]:
def prod1_profile_text(prod1_cat2_prefer, prod1_cat1_prefer, prod1_item_prefer):
    if prod1_item_prefer == "정보없음":
        return ""
    else:
        return f"저는 {prod1_cat2_prefer} 통해서 요금제를 선택했고, 구체적으로 {prod1_cat1_prefer} 중 {prod1_item_prefer} 요금제를 쓰고 있어요."

prod1_text_udf = udf(prod1_profile_text, StringType())

In [11]:
total_data1 = total_data1.withColumn("prod1_text", prod1_text_udf(total_data1["prod1_cat2_prefer"], total_data1["prod1_cat1_prefer"], total_data1["prod1_item_prefer"]))

- subsc : 구독하고 있는 상품 정보

In [12]:
def subsc_profile_text(subsc_item_prefer):
    if subsc_item_prefer == "정보없음":
        return ""
    else:
        return f"저는 구독 상품 중에서 {subsc_item_prefer}를 사용하고 있어요."

subsc_text_udf = udf(subsc_profile_text, StringType())

In [13]:
total_data1 = total_data1.withColumn("subsc_text", subsc_text_udf(total_data1["subsc_item_prefer"]))

- tdeal : 티딜

In [14]:
def tdeal_profile_text(tdeal_cat2_prefer, tdeal_item_prefer):
    if tdeal_item_prefer == "정보없음":
        return ""
    else:
        return f"저는 티딜 쇼핑몰에서 주로 {tdeal_cat2_prefer} 카테고리에 관심이 있고, 최근에는 {tdeal_item_prefer} 키워드 기반의 상품을 찾아봤어요."

tdeal_text_udf = udf(tdeal_profile_text, StringType())

In [15]:
total_data1 = total_data1.withColumn("tdeal_text", tdeal_text_udf(total_data1["tdeal_cat2_prefer"], total_data1["tdeal_item_prefer"]))

- tmap

In [16]:
def tmap_profile_text(tmap_cat2_prefer, tmap_item_prefer):
    if tmap_item_prefer == "정보없음":
        return ""
    else:
        return f"저는 운전할 때 주로 {tmap_cat2_prefer} 관련된 카테고리를 입력 후 이동하며, 최근에는 {tmap_item_prefer} 키워드 관련 경로지로 주로 이동했어요."

tmap_text_udf = udf(tmap_profile_text, StringType())

In [17]:
total_data1 = total_data1.withColumn("tmap_text", tmap_text_udf(total_data1["tmap_cat2_prefer"], total_data1["tmap_item_prefer"]))

- tmbr : 티멤버십

In [18]:
def tmbr_profile_text(tmbr_item_prefer):
    if tmbr_item_prefer == "정보없음":
        return ""
    else:
        return f"저는 티멤버십에서 {tmbr_item_prefer} 혜택을 주로 사용하고 있어요."

tmbr_text_udf = udf(tmbr_profile_text, StringType())

In [19]:
total_data1 = total_data1.withColumn("tmbr_text", tmbr_text_udf(total_data1["tmbr_item_prefer"]))

- xdr : 앱 사용 정보

In [20]:
def xdr_profile_text(xdr_cat2_prefer, xdr_item_prefer):
    if xdr_item_prefer == "정보없음":
        return ""
    else:
        return f"제가 주로 사용하는 앱의 카테고리는 {xdr_cat2_prefer} 들이 있고, 최근에는 {xdr_item_prefer} 앱을 사용했어요."

xdr_text_udf = udf(xdr_profile_text, StringType())

In [21]:
total_data1 = total_data1.withColumn("xdr_text", xdr_text_udf(total_data1["xdr_cat2_prefer"],total_data1["xdr_item_prefer"]))

- xtr : MNO 서비스 관련
    - item : 사용한 서비스
    - cat1 : item 상위 레벨
    - cat2 : 이용채널

In [22]:
def xtr_profile_text(xtr_cat2_prefer, xtr_cat1_prefer, xtr_item_prefer):
    if xtr_item_prefer == "정보없음":
        return ""
    else:
        return f"저는 주로 MNO(통신) 관련 서비스를 이용하는 채널로 {xtr_cat2_prefer}를 이용하고 있고, 자주 사용하는 서비스는 {xtr_cat1_prefer}이고 최근 {xtr_item_prefer} 기능을 많이 사용하고 있어요."

xtr_text_udf = udf(xtr_profile_text, StringType())

In [23]:
total_data1 = total_data1.withColumn("xtr_text", xtr_text_udf(total_data1["xtr_cat2_prefer"],total_data1["xtr_cat1_prefer"],total_data1["xtr_item_prefer"]))

### (2) Adot 도메인 TEXT 화
- 메타 정보는 음악만 풍부한 상황이기 때문에 음악만 따로 자세하게 구별해서 텍스트화

In [24]:
sevice_set = {'apollo_alarm':"알람", "apollo_anniversary":"생일", "apollo_calendar":"캘린더", "apollo_calendar2":"특일정보", "apollo_call":"전화", "apollo_campaign":"캠페인", "apollo_character":"캐릭터꾸미기",
              "apollo_chatgpt":"챗T", "apollo_congestion":"혼잡도", "apollo_cqa":"CQA", "apollo_dailycostume":"오늘뭐입지", "apollo_dictionary":"어학사전", "apollo_english":"튜터", "apollo_fortune":"운세",
              "apollo_friendsbitna":"프렌즈_길빛나", "apollo_friendsharu":"프렌즈_강하루", "apollo_friendsj":"프렌즈_육제이", "apollo_game":"게임", "apollo_glm":"감성대화", "apollo_glmtosqa":"GLMtoSQA", "apollo_keep":"keep",
              "apollo_liveqa":"LiveQA", "apollo_menu":"메뉴추천", "apollo_message":"문자", "apollo_mj":"맛집추천", "apollo_mno":"T서비스", "apollo_music":"음악", "apollo_mytv":"TV", "apollo_news":"뉴스", "apollo_photo":"포토",
              "apollo_podcast":"팟캐스트", "apollo_qfeed":"큐피드", "apollo_quest":"퀘스트", "apollo_radio":"라디오", "apollo_recipe":"레시피", "apollo_reward":"리워드", "apollo_ring":"링", "apollo_routine":"루틴",
              "apollo_samsungstock":"증권", "apollo_sleep":"sleep", "apollo_sports":"스포츠", "apollo_sqa":"SQA", "apollo_survey":"선호도조사", "apollo_time":"시간", "apollo_tmap":"TMAP", "apollo_tmembership":"T 멤버십",
              "apollo_tworld":"T 월드", "apollo_video":"비디오", "apollo_weather":"날씨", "apollo_wordchain":"끝말잇기", "apollo_friends":"프렌즈톡", "apollo_media":"미디어", "apollo_qa":"대표발화"}

In [25]:
adot_cat1_data = bq_to_df("""
    SELECT user_id, adot_cat1_prefer as adot_cat1
    FROM x1113099.one_model_profile_adot_cat1
    WHERE dt = (SELECT max(dt) FROM x1113099.one_model_profile_adot_cat1)
""")

adot_cat3_item_data = bq_to_df("""
    SELECT *
    FROM x1113099.one_model_profile_adot_cat3_item
    WHERE dt = (SELECT max(dt) FROM x1113099.one_model_profile_adot_cat3_item)
""")

24/06/18 15:29:54 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
24/06/18 15:30:01 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


In [26]:
adot_merged = adot_cat1_data.join(adot_cat3_item_data, on=['user_id','adot_cat1'],how="left")

In [27]:
def adot_domain_to_text(adot_cat1):
    return sevice_set[f'{adot_cat1}']

adot_domain_to_text_udf = udf(adot_domain_to_text, StringType())
adot_merged = adot_merged.withColumn("adot_cat1_text", adot_domain_to_text_udf(adot_merged["adot_cat1"]))

In [28]:
adot_merged1 = adot_merged.groupBy("user_id").agg(concat_ws(",",collect_list('adot_cat1_text')).alias(f'adot_cat1_text_list'),
                                                  concat_ws(",",collect_list('adot_item_prefer')).alias(f'adot_item_prefer_list'),
                                                  concat_ws(",",collect_list('music_item_list')).alias(f'music_item_list'),
                                                  concat_ws(",",collect_list('adot_cat3_prefer')).alias(f'adot_cat3_prefer_list'))

adot_merged2 = adot_merged.groupBy("user_id").agg(concat_ws(",",collect_list('cat3_genre_list')).alias(f'cat3_genre_list'),
                                                  concat_ws(",",collect_list('cat3_artist_list')).alias(f'cat3_artist_list'))

In [29]:
final_adot_merged = adot_merged1.join(adot_merged2, on="user_id", how="left")

In [30]:
final_adot_merged.show(n=10)

+--------------------+-----------------------------+---------------------+------------------------+---------------------+---------------+-----------------------------+
|             user_id|          adot_cat1_text_list|adot_item_prefer_list|         music_item_list|adot_cat3_prefer_list|cat3_genre_list|             cat3_artist_list|
+--------------------+-----------------------------+---------------------+------------------------+---------------------+---------------+-----------------------------+
|APL00000BJ0H4R30LSLC|                         루틴|                     |                        |                     |               |                             |
|APL00000BJ19IFQXK54W|                         전화|                     |                        |                     |               |                             |
|APL00000BJ1GGZ9T2MM8|음악,챗T,맛집추천,뉴스,Liv...|                     |                        |                     |               |                             |
|APL0

In [31]:
def adot_profile_text(adot_cat1_text_list, adot_item_prefer_list, music_item_list, adot_cat3_prefer_list, cat3_genre_list, cat3_artist_list):
    if "음악" in adot_cat1_text_list:
        if (adot_item_prefer_list != "") & (adot_cat3_prefer_list != ""):
            return f"저는 에이닷(AI Agent Service) 서비스 기능 중 {adot_cat1_text_list} 서비스를 사용하고 있고, 주로 사용하는 컨텐츠는 {adot_cat3_prefer_list}이고 최근에는 {adot_item_prefer_list}에 관심이 있습니다. \
                     그리고 특히 음악 서비스 중에서는 {cat3_genre_list} 장르와 {cat3_artist_list} 가수를 좋아하고 최근에는 {music_item_list} 곡을 들었어요."
        else:
            return f"저는 에이닷(AI Agent Service) 서비스 기능 중 {adot_cat1_text_list} 서비스를 사용하고 있습니다. \
                     그리고 특히 음악 서비스 중에서는 {cat3_genre_list} 장르와 {cat3_artist_list} 가수를 좋아하고 최근에는 {music_item_list} 곡을 들었어요."
            
    else:
        if (adot_item_prefer_list != "") & (adot_cat3_prefer_list != ""):
            return f"저는 에이닷(AI Agent Service) 서비스 기능 중 {adot_cat1_text_list} 서비스를 사용하고 있고, 주로 사용하는 컨텐츠는 {adot_cat3_prefer_list}이고 최근에는 {adot_item_prefer_list}에 관심이 있습니다."
        else:
            return f"저는 에이닷(AI Agent Service) 서비스 기능 중 {adot_cat1_text_list} 서비스를 사용하고 있습니다."

adot_text_udf = udf(adot_profile_text, StringType())

In [32]:
final_adot_merged = final_adot_merged.withColumn("adot_text", adot_text_udf(final_adot_merged["adot_cat1_text_list"],final_adot_merged["adot_item_prefer_list"],final_adot_merged["music_item_list"],
                                                                            final_adot_merged["adot_cat3_prefer_list"],final_adot_merged["cat3_genre_list"],final_adot_merged["cat3_artist_list"]))

### (3) Merge

In [42]:
final_total_data = final_adot_merged.join(total_data1, total_data1.luna_id==final_adot_merged.user_id, how="left")

- 추가로 나이와 성별에 대한 text 화

In [43]:
def demo_profile_text(gender, age):
    if gender == "female":
        return f"저의 성별을 여성이고, 나이는 {age}살 입니다."
    else: 
        return f"저의 성별을 남성이고, 나이는 {age}살 입니다."

demo_text_udf = udf(demo_profile_text, StringType())

In [44]:
final_total_data = final_total_data.withColumn("demo_text", demo_text_udf(final_total_data["gender"],final_total_data["age"]))

In [45]:
final_total_data_text = final_total_data.select(['svc_mgmt_num','user_id','_11st_text','cdr_text','prod1_text','subsc_text','tdeal_text','tmap_text','tmbr_text','xdr_text','xtr_text','adot_text','demo_text'])
final_total_data_text = final_total_data_text.withColumn("dt", lit(DT).cast("date"))

In [46]:
# final_total_data_text.count()

In [47]:
final_total_data_text.printSchema()

root
 |-- svc_mgmt_num: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- _11st_text: string (nullable = true)
 |-- cdr_text: string (nullable = true)
 |-- prod1_text: string (nullable = true)
 |-- subsc_text: string (nullable = true)
 |-- tdeal_text: string (nullable = true)
 |-- tmap_text: string (nullable = true)
 |-- tmbr_text: string (nullable = true)
 |-- xdr_text: string (nullable = true)
 |-- xtr_text: string (nullable = true)
 |-- adot_text: string (nullable = true)
 |-- demo_text: string (nullable = true)
 |-- dt: date (nullable = true)



### (4) Text화 된 데이터만 AIDP 업로드

In [48]:
dest_dataset = "x1113099"
partitioned_dest_table = "one_model_profile_text"

In [49]:
get_bigquery_client().query(f"""
    CREATE TABLE IF NOT EXISTS {dest_dataset}.{partitioned_dest_table}
    (
        svc_mgmt_num STRING,
        user_id STRING,
        _11st_text STRING,
        cdr_text STRING,
        prod1_text STRING,
        subsc_text STRING,
        tdeal_text STRING,
        tmap_text STRING,
        tmbr_text STRING,
        xdr_text STRING,
        xtr_text STRING,
        adot_text STRING,
        demo_text STRING,
        dt DATE,
    )
    PARTITION BY dt
""").result()

print(f"생성된 테이블 : {dest_dataset}.{partitioned_dest_table}")

생성된 테이블 : x1113099.one_model_profile_text


In [50]:
df_to_bq_table(df=final_total_data_text,
               dataset=dest_dataset,
               table_name=partitioned_dest_table,
               mode="overwrite")